In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibiilty
torch.manual_seed(1)

In [3]:
x_data = [[1, 2, 1],
          [1, 3, 2],
          [1, 3, 4],
          [1, 5, 3],
          [1, 7, 5],
          [1, 2, 5],
          [1, 6, 6],
          [1, 7, 7]]
y_data = [[2], [2], [2], [1], [1], [1], [0], [0]]

In [4]:
x_test = [[2, 1, 1], [3, 1, 2], [3, 3, 4]]
y_test = [[2], [2], [2]]

In [5]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 3)
        
    def forward(self, x):
        return self.linear(x)

In [6]:
from torch.utils.data import DataLoader, Dataset

class TrainData(Dataset):
    def __init__(self):
        self.x = x_data
        self.y = y_data
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x[idx])
        y = torch.LongTensor(self.y[idx])
        
        return x, y
    
class TestData(Dataset):
    def __init__(self):
        self.x = x_test
        self.y = y_test
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x[idx])
        y = torch.LongTensor(self.y[idx])
        
        return x, y

In [7]:
train_dataset = TrainData()
test_dataset  = TestData()

In [8]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=.1)

In [9]:
train_dataloader = DataLoader(train_dataset,
                              batch_size=2,
                              shuffle=True
                             )

In [10]:
from IPython.display import clear_output

def train(model, optimizer):
    nb_epochs = 100
    for epoch in range(nb_epochs + 1):
        for batch_num, samples in enumerate(train_dataloader):
            x, y = samples
            y.squeeze_(-1)
            
            predictions = model(x)
            cost = F.cross_entropy(predictions, y)
            
            optimizer.zero_grad()
            cost.backward()
            optimizer.step()
            
            clear_output(wait=True)
            print("Epoch {:4d}/{} Batch {:2d}/{} Cost: {:.6f}".format(
                epoch, nb_epochs, batch_num+1, len(train_dataset), cost.item()
            ))

In [11]:
def test(model, optimizer, x_test, y_test):
    x_test = torch.FloatTensor(x_test)
    y_test = torch.LongTensor(y_test)
    y_test.squeeze_(-1)
    
    prediction = model(x_test)
    predicted_classes = prediction.max(1)[1]

    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction, y_test)
    
    print("Acc: {}% Cost: {:.6f}".format(
        correct_count / len(y_test) * 100, cost.item()
    ))

In [12]:
train(model, optimizer)

Epoch  100/100 Batch  4/8 Cost: 0.420669


In [13]:
test(model, optimizer, x_test, y_test)

Acc: 100.0% Cost: 0.051542
